In [11]:
import os
from sklearn import manifold
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
import sys
from pathlib import Path
path = str(Path(os.getcwd()).parent)
sys.path.insert(0, path)
from breakout.vae.vae import ConvVAE

import_path = str(Path(os.getcwd()).parent)

sys.path.insert(0, import_path)

from breakout.model import make_model, _process_frame

/home/dakaun/PycharmProjects/world_model/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/dakaun/PycharmProjects/world_model/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/dakaun/PycharmProjects/world_model/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/dakaun/

In [6]:
num_actions = 4
    
gamename ='breakout'
optimizer = 'cma'
num_episode = 16
population = 32 # = num_workers * num_worker_trial
    
path = '/home/dakaun/Dropbox/MA/worldmodel/worldmodel-breakout-server-version-v3/200420'
    
file_path = path + '/log/'
file_base = gamename+'.'+optimizer+'.'+str(num_episode)+'.'+str(population)
    
vae_path = path + '/tf_vae'
rnn_path = path + '/tf_rnn'
    
record = path + '/record'

In [7]:
model = make_model(rnn_path=os.path.join(rnn_path, 'rnn.json'), vae_path=os.path.join(vae_path, 'vae.json'))

model.load_model(os.path.join(file_path, file_base + '.best.json'))

INFO:tensorflow:Model using cpu.
loading file /home/dakaun/Dropbox/MA/worldmodel/worldmodel-breakout-server-version-v3/200420/log/breakout.cma.16.32.best.json


In [8]:
def simulate(model, train_mode=False, render_mode=True, num_episode=5, seed=-1, max_len=-1):
    reward_list = []
    t_list = []
    max_episode_length = 2100
    tsne_data = pd.DataFrame()    
    action_list = []#action_list_episode = []
    observation_list_episode = []
    reward_target = []
    
    if (seed >= 0):
        random.seed(seed)
        np.random.seed(seed)
        model.env.seed(seed)
    

    for episode in range(num_episode):
        #action_list = []
        observation_list = []
        obs = model.env.reset()
        if obs is None:
            obs = deepcopy(model.env.reset())
        #obs = _process_frame(obs)

        total_reward = 0.0

        for t in range(max_episode_length):
            if render_mode:
                model.env.render("human")
                model.env.unwrapped.viewer.window.on_key_press = key_press
                if RENDER_DELAY:
                    time.sleep(0.01)
            else:
                model.env.render('rgb_array')
            obs = _process_frame(obs)
            z, mu, logvar = model.encode_obs(obs)
            _, action = model.get_action(z)
            obs, reward, done, info = model.env.step(action)

            action_list.append(int(action))
            #observation_list.append(obs)
            #obs = _process_frame(obs)
            total_reward += reward
            reward_target.append(int(reward))
            
            data = np.concatenate([z, model.state.h[0]]).reshape(1,288)
            #data = np.append(data, action).reshape(1,289)
            tsne_data = tsne_data.append(pd.DataFrame(data), ignore_index=True)

            if done:
                if render_mode:
                    model.env.close()
                #action_list_episode.append(np.array(action_list))
                #observation_list_episode.append(observation_list)
                break
        if render_mode:
            print("reward", total_reward, "timesteps", t)
        #reward_list.append(total_reward)
        #t_list.append(t)
    return action_list, tsne_data, reward_target


In [9]:
actions, tsne_data, reward_target= simulate(model, train_mode=False, render_mode=False, num_episode=50)

In [15]:
from sklearn.decomposition import PCA

In [35]:
pca = PCA(n_components=4)

In [36]:
pca_tsne_data = pca.fit_transform(tsne_data)

In [37]:
pca.explained_variance_ratio_

array([0.1467, 0.0951, 0.0466, 0.0333])

In [38]:
sum(pca.explained_variance_ratio_)

0.3217422791062381

In [33]:
for i in range(10,30):
    print(i)
    pca = PCA(n_components=i)
    pca.fit_transform(tsne_data)
    
    print(pca.explained_variance_ratio_)
    
    print(sum(pca.explained_variance_ratio_))
    print('\n')

10
[0.1467 0.0951 0.0466 0.0332 0.0263 0.0251 0.0246 0.0243 0.0242 0.0242]
0.47040713864023725


11
[0.1467 0.0951 0.0466 0.0333 0.0265 0.0251 0.0246 0.0241 0.0241 0.024  0.0237]
0.49373231205738843


12
[0.1467 0.0951 0.0466 0.0334 0.0262 0.0251 0.0246 0.0244 0.0242 0.0241 0.0238 0.0237]
0.517881528535452


13
[0.1467 0.0951 0.0466 0.0333 0.0263 0.0253 0.0248 0.0244 0.0242 0.024  0.0238 0.0236 0.0236]
0.5418776934509211


14
[0.1467 0.0951 0.0466 0.0334 0.0265 0.0254 0.0246 0.0244 0.0242 0.024  0.0238 0.0237 0.0236 0.0234]
0.5653197527894545


15
[0.1467 0.0951 0.0466 0.0334 0.0263 0.0253 0.0248 0.0244 0.0243 0.0241 0.0238 0.0237 0.0237 0.0236
 0.0234]
0.5893024645333006


16
[0.1467 0.0951 0.0466 0.0333 0.0264 0.0254 0.0247 0.0245 0.0243 0.0242 0.0239 0.0238 0.0237 0.0236
 0.0234 0.0233]
0.6129804276063091


17
[0.1467 0.0951 0.0466 0.0333 0.0265 0.0254 0.0248 0.0245 0.0243 0.0242 0.0239 0.0238 0.0237 0.0236
 0.0233 0.0233 0.0231]
0.6362250613044952


18
[0.1467 0.0951 0.0466 0.0334 

In [41]:
abs(pca.components_) # shape 4,288

array([[0.0099, 0.049 , 0.0041, 0.0023, 0.0021, 0.0035, ..., 0.0445, 0.0396, 0.0111, 0.0459,
        0.0231, 0.0156],
       [0.0123, 0.317 , 0.011 , 0.0229, 0.0122, 0.0023, ..., 0.0027, 0.0697, 0.0108, 0.0171,
        0.0449, 0.0319],
       [0.0004, 0.0592, 0.0228, 0.0018, 0.0022, 0.018 , ..., 0.0063, 0.0468, 0.029 , 0.0564,
        0.0829, 0.0818],
       [0.0337, 0.2857, 0.0687, 0.0133, 0.0227, 0.0867, ..., 0.0536, 0.006 , 0.0307, 0.0194,
        0.0064, 0.0396]])

In [49]:
abs(pca.components_[0])

array([0.0099, 0.049 , 0.0041, 0.0023, 0.0021, 0.0035, 0.0089, 0.031 , 0.0059, 0.013 , 0.0255,
       0.088 , 0.001 , 0.0057, 0.0087, 0.0016, 0.0016, 0.0107, 0.0052, 0.0018, 0.012 , 0.3192,
       0.0197, 0.0087, 0.0025, 0.3645, 0.0082, 0.1128, 0.1982, 0.0016, 0.0066, 0.1481, 0.0033,
       0.0443, 0.0252, 0.0705, 0.0136, 0.0126, 0.0105, 0.094 , 0.0547, 0.0055, 0.0527, 0.0053,
       0.0709, 0.0065, 0.0189, 0.0445, 0.0074, 0.0929, 0.024 , 0.1899, 0.0123, 0.0096, 0.0874,
       0.0562, 0.1024, 0.0515, 0.0215, 0.0224, 0.0192, 0.0417, 0.0144, 0.0633, 0.025 , 0.0506,
       0.0372, 0.0075, 0.0439, 0.0347, 0.0179, 0.0224, 0.0325, 0.0533, 0.0307, 0.0194, 0.0281,
       0.0002, 0.0886, 0.0221, 0.0975, 0.0062, 0.0245, 0.062 , 0.0379, 0.0104, 0.0253, 0.054 ,
       0.0479, 0.0096, 0.0134, 0.0408, 0.0653, 0.0431, 0.0194, 0.0481, 0.0067, 0.0201, 0.0314,
       0.0491, 0.0288, 0.105 , 0.1091, 0.0331, 0.0338, 0.0199, 0.0629, 0.069 , 0.0298, 0.0324,
       0.0015, 0.0248, 0.0702, 0.1325, 0.0391, 0.0

In [48]:
np.where(abs(pca.components_[0]) == np.amax(abs(pca.components_[0])))

(array([25]),)